# Here assignment

In [14]:
import requests
import pandas as pd
APP_API_KEY='th_OeLpIUsmB4JbVDqWAVYRuVKilr3Pan2ZzKD9r604'

In [15]:
# For collecting 10 address from 10 cities each. Total 100 address
# Googled lat/lng coordinates of each city as below 
# This block will search 10 petrol pumps in each city and save that address in text file.

city_dict={
'Delhi':['28.6959','77.13725'],
'Mumbai':['19.0760','72.8777'],
'Pune':['18.5204','73.8567'],
'Bengaluru':['12.9716','77.5946'],
'Chennai':['13.0827','80.2707'],
'Chandigarh':['30.7333','76.7794'],
'Kolkata':['22.5726','88.3639'],
'Raipur':['21.2514','81.6296'],
'Jodhapur':['26.2389','73.0243'],
'Hyderabad':['17.3850','78.4867']
}  

for city,cordinates in city_dict.items():
    query='https://discover.search.hereapi.com/v1/discover?at='+cordinates[0]+','+cordinates[1]\
    +'&q=petrol+station&limit=10&apiKey='+APP_API_KEY

    res=requests.get(query).json()
    
    for i in range(len(res['items'])): #10 results
        address=res['items'][i]['address']['label']
        #cityname=res['items'][i]['address']['city']
        with open('sample_address.txt','a')as f:
            f.write(city+'|'+address+'\n')
    print('file saved')

file saved
file saved
file saved
file saved
file saved
file saved
file saved
file saved
file saved
file saved


In [16]:
# To flatten out the respone with nested lists

def make_flat_list(result):
    ans=[]
    for k,v in result.items():
        if isinstance(v,list):    
            for ele in v:
                ans.append(ele)
        else:
            ans.append(v)
    return ans

In [17]:
df=pd.DataFrame(columns=['address','city','accuracy','resultType','lat','lng','countryCode','stateCode','postalCode'])
df

,address,city,accuracy,resultType,lat,lng,countryCode,stateCode,postalCode


In [18]:
geocode_endpoint='https://geocode.search.hereapi.com/v1/geocode?q='
filename='sample_address.txt'
with open(filename,'r') as f:
    address=f.readlines()

for addr in address:
    new_record=[]
    addr=addr.replace('\n','')
    new_record.append(addr)
    cityname=addr.split('|')[0]
    addr=addr.split('|')[1]
    addr=addr.replace(',','%2C')
    addr=addr.replace(' ','+')
    geoquery=geocode_endpoint+addr+'&apiKey='+APP_API_KEY
    res=requests.get(geoquery).json()

    flat_score_lst=make_flat_list(res['items'][0]['scoring']['fieldScore'])
    field_count=len(flat_score_lst)    
    accuracy=sum(flat_score_lst)/field_count * 100
    
    new_record.append(cityname)
    new_record.append(round(accuracy,2))
    
    new_record.append(res['items'][0]['resultType'])
    new_record.append(res['items'][0]['position']['lat'])
    new_record.append(res['items'][0]['position']['lng'])
    new_record.append(res['items'][0]['address']['countryCode'])
    new_record.append(res['items'][0]['address']['stateCode'])
    new_record.append(res['items'][0]['address']['postalCode']) 

    df.loc[len(df.index)] = new_record

    #print('Detected city:{} with accuracy of {}'.format(cityname,round(accuracy,2)))
print('done')

done


In [19]:
df.head()

,address,city,accuracy,resultType,lat,lng,countryCode,stateCode,postalCode
0,"Delhi|Indian Oil, Pitampura, Delhi 110034, India",Delhi,100.00,locality,28.69590,77.13725,IND,DL,110034
1,"Delhi|Hindustan Petroleum, Mall Road, Block J,...",Delhi,100.00,place,28.69189,77.14671,IND,DL,110034
2,"Delhi|Hindustan Petroleum, Mall Road, Block J,...",Delhi,100.00,place,28.69189,77.14671,IND,DL,110034
3,"Delhi|Vidya Sagar Gas Agency, A-16, Pitampura ...",Delhi,100.00,place,28.70000,77.12783,IND,DL,110034
4,"Delhi|Hindustan Petroleum, महाराजा अग्रसेन मार...",Delhi,98.57,place,28.70505,77.14088,IND,DL,110034


In [20]:
df.to_csv('city_details.csv')
print('done')

done
